# Elastic engine demo

In [1]:
import datalabframework as dlf
dlf.project.info()

{'workrun': 'default',
 'filename': 'elastic.ipynb',
 'rootpath': '/home/thuc/teko/product-search/dlf-demos_fork/demos/elastic',
 'workdir': '/home/thuc/teko/product-search/dlf-demos_fork/demos/elastic'}

In [4]:
metadata = dlf.params.metadata()
dlf.utils.pretty_print(metadata)
# just the resources, some pretty printing
dlf.utils.pretty_print(metadata['resources'])

engines:
  elastic:
    context: elastic
  spark-local:
    config:
      jobname: default
      master: local[*]
    context: spark
loggers:
  stream:
    enable: true
    severity: info
providers:
  elastic_test:
    hostname: 123.31.32.226
    port: 9200
    service: elastic
  local_filesystem:
    format: csv
    path: data
    service: local
resources:
  .keywords:
    path: keywords.csv
    provider: local_filesystem
    read:
      mapping:
        abc:
          drop: true
          fields:
            keyword:
              ignore_above: 256
              type: keyword
          fill: value
          name: cds
          type: text
      options:
        header: true
        inferSchema: true
  .keywords_elk:
    index: search_keywords_dev
    mappings:
      doc_type: keyword
      properties:
        api_result: integer
        count: integer
        gg: integer
        gg_clicks: double
        gg_impressions: double
        gg_position: double
        pv: integer
        pv

In [4]:
import datalabframework as dlf
elastic_engine = dlf.engines.get('elastic')
elastic_engine.info

{'name': 'elastic', 'context': 'elastic', 'config': {}}

In [5]:
# dlf.data.metadata('keywords_elk')

In [13]:
res = elastic_engine.read(".keywords_search", _query="laptop", _from=0, _size=20)
res

Summary: {'took': 1, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'total_hits': 356, 'max_score': 645.3939}


,_id,_index,_score,_type,api_result,count,gg,gg_impressions,gg_position,impressions,position,pv,pv_result,query,query_wo_tones
0,b8ohU2YBknddiPsCn8wc,search_keywords_dev,645.39390,keyword,966.0,7,1,253461.0,4.763131,33391.000000,1.0,1,726.0,laptop,laptop
1,CsoqU2YBknddiPsCGN0b,search_keywords_dev,645.39390,keyword,966.0,7,1,253461.0,4.763131,33391.000000,1.0,1,726.0,laptop,laptop
2,DsohU2YBknddiPsCn9Fq,search_keywords_dev,473.32490,keyword,1452.0,7,1,115890.0,4.356658,26945.000000,1.0,1,119.0,laptop dell,laptop dell
3,WcoqU2YBknddiPsCGOJd,search_keywords_dev,473.32490,keyword,1452.0,7,1,115890.0,4.356658,26945.000000,1.0,1,119.0,laptop dell,laptop dell
4,MMohU2YBknddiPsCn8wc,search_keywords_dev,414.95386,keyword,157.0,2,1,67170.0,6.588968,20709.000000,1.0,1,147.0,laptop asus,laptop asus
5,yMoqU2YBknddiPsCGNwb,search_keywords_dev,414.95386,keyword,157.0,2,1,67170.0,6.588968,20709.000000,1.0,1,147.0,laptop asus,laptop asus
6,1cohU2YBknddiPsCn80e,search_keywords_dev,293.26788,keyword,1744.0,3,1,16525.0,1.677919,10344.000000,1.0,1,56.0,ram laptop,ram laptop
7,pMoqU2YBknddiPsCGN8f,search_keywords_dev,293.26788,keyword,1744.0,3,1,16525.0,1.677919,10344.000000,1.0,1,56.0,ram laptop,ram laptop
8,2MohU2YBknddiPsCn8oc,search_keywords_dev,236.21849,keyword,134.0,2,1,32419.0,7.306636,6711.000000,1.0,1,144.0,laptop hp,laptop hp
9,MsoqU2YBknddiPsCGNob,search_keywords_dev,236.21849,keyword,134.0,2,1,32419.0,7.306636,6711.000000,1.0,1,144.0,laptop hp,laptop hp


In [12]:
type(res)

pandas.core.frame.DataFrame

In [3]:
elastic_engine.write([{"query":"laptop", "query_wo_tones":"laptop"}, {"query":"màn hình", "query_wo_tones":"man hinh"}], ".keywords_elk", mode='overwrite')

{'api_result': {'type': 'integer'}, 'count': {'type': 'integer'}, 'gg': {'type': 'integer'}, 'gg_clicks': {'type': 'double'}, 'gg_impressions': {'type': 'double'}, 'gg_position': {'type': 'double'}, 'pv': {'type': 'integer'}, 'pv_impressions': {'type': 'double'}, 'pv_position': {'type': 'double'}, 'pv_result': {'type': 'double'}, 'query': {'fields': {'keyword': {'ignore_above': 256, 'type': 'keyword'}}, 'type': 'text'}, 'query_wo_tones': {'fields': {'keyword': {'ignore_above': 256, 'type': 'keyword'}}, 'type': 'text'}}


In [10]:
engine = dlf.engines.get('spark-local')

PYSPARK_SUBMIT_ARGS:  pyspark-shell


In [11]:
df = engine.read(".keywords")

repartition  None
coalesce  None
cache False
file:///home/thuc/teko/product-search/dlf-demos_fork/demos/elastic/data/keywords.csv


In [ ]:
df.show()

In [12]:
elastic_engine.write(df, ".keywords_elk", mode='append')

In [3]:
import pandas as pd
pddf = pd.read_csv("data/keywords.csv")
# pddf

In [8]:
elastic_engine.write(pddf, ".keywords_elk", mode='overwrite')

{'index': {'mapping': {'total_fields': {'limit': 1024}}, 'number_of_replicas': 3, 'number_of_shards': 1}}
{'api_result': {'type': 'integer'}, 'count': {'type': 'integer'}, 'gg': {'type': 'integer'}, 'gg_clicks': {'type': 'double'}, 'gg_impressions': {'type': 'double'}, 'gg_position': {'type': 'double'}, 'pv': {'type': 'integer'}, 'pv_impressions': {'type': 'double'}, 'pv_position': {'type': 'double'}, 'pv_result': {'type': 'double'}, 'query': {'fields': {'keyword': {'ignore_above': 256, 'type': 'keyword'}}, 'type': 'text'}, 'query_wo_tones': {'fields': {'keyword': {'ignore_above': 256, 'type': 'keyword'}}, 'type': 'text'}}


In [13]:
a ={'api_result': 'double',
   'count': 'integer',
   'gg': 'integer',
   'gg_clicks': 'double',
   'gg_impressions': 'double',
   'gg_position': 'double',
   'pv': 'integer',
   'pv_impressions': 'double',
   'pv_position': 'double',
   'pv_result': 'double',
   'query': {'fields': {'keyword': {'ignore_above': 256, 'type': 'keyword'}},
    'type': 'text'},
   'query_wo_tones': {'fields': {'keyword': {'ignore_above': 256,
      'type': 'keyword'}},
    'type': 'text'}}
type(a)

dict

In [14]:
type("")

str